In [5]:

import s3fs
import pandas
import polars as pl
import pyarrow.dataset as ds

fs = s3fs.S3FileSystem(profile="aws_profile_name")
bucket = "<S3_BUCKET_NAME>"
path = "<S3_PATH>/partition=<S3_PARTITION>"

s3_endpoint = f"s3://{bucket}/{path}"

myds = ds.dataset([y for y in fs.ls(s3_endpoint) if ".parquet" in y], filesystem=fs)
lazy_df = pl.scan_ds(myds)

df = lazy_df.select(pl.col('*')).collect()

print(df)

AttributeError: 'S3' object has no attribute 'Session'

In [ ]:
import boto3
import polars as pl


# Your MinIO/S3 credentials and bucket details
endpoint_url = 'http://localhost:9000'  # Your MinIO endpoint
access_key = 'minio_debug'
secret_key = 'minio123'
bucket_name = 'depictio-bucket'
object_key = '64a842842bf4fa7deaa3dbed'  # The name of the file you want to read

# Create an S3 client
s3_client = boto3.client('s3',
                         endpoint_url=endpoint_url,
                         aws_access_key_id=access_key,
                         aws_secret_access_key=secret_key,
                         region_name='us-east-1')  # Region name can be any string

session = boto3.session.Session()
credentials = session.get_credentials()

df = pl.scan_parquet(
    "s3://coiled-data/uber/*.parquet",
    storage_options={
        "aws_access_key_id": credentials.access_key,
        "aws_secret_access_key": credentials.secret_key,
        "region": "us-east-2",
        "session_token": credentials.token,
    },
)

In [1]:
import os

# Set environment variables for MinIO access
# os.environ["AWS_ACCESS_KEY_ID"] = "DSj0vuC7bfMUGZpX2L4h"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "wQ4OGBfXDwBV0nXuW0cHQfPqSDfhMOKUqi3kjtLU"
os.environ["AWS_ACCESS_KEY_ID"] = "DSj0vuC7bfMUGZpX2L4h"
os.environ["AWS_SECRET_ACCESS_KEY"] = "wQ4OGBfXDwBV0nXuW0cHQfPqSDfhMOKUqi3kjtLU"
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_REGION"] = "us-east-1"

import polars as pl

# Now attempt to read the Delta table
df = pl.read_delta("s3://depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0")
df


OSError: Generic S3 error: Client error with status 403 Forbidden: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>InvalidAccessKeyId</Code><Message>The AWS Access Key Id you provided does not exist in our records.</Message><AWSAccessKeyId>DSj0vuC7bfMUGZpX2L4h</AWSAccessKeyId><RequestId>SMJJE2JNPAQMWSAX</RequestId><HostId>ZlPsIp4cYlVawh1iIJw1MqRwqYKLD5Yn3bwuQnAyz9T7R6zt2M5+W1L1rqoetln8ETSfUx97cGQ=</HostId></Error>

In [4]:
import s3fs
fs = s3fs.S3FileSystem()  
fs.

In [14]:
import polars as pl
import s3fs

storage_options = {
    "AWS_ACCESS_KEY_ID": "minio",
    "AWS_SECRET_ACCESS_KEY": "minio123",
    "AWS_S3_ENDPOINT_URL": "http://localhost:9000",
    "AWS_REGION": "us-east-1",
}
conn_data = { 'key': 'minio', 'secret': 'minio123', 'client_kwargs': { 'endpoint_url': 'http://localhost:9000' }}
fs = s3fs.S3FileSystem(**conn_data)
fs
df = pl.read_delta("http://localhost:9000/depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0")
df


thread '<unnamed>' panicked at /Users/runner/work/delta-rs/delta-rs/crates/deltalake-core/src/table/builder.rs:158:48:
The specified table_uri is not valid: InvalidTableLocation("Unknown scheme: http")


PanicException: The specified table_uri is not valid: InvalidTableLocation("Unknown scheme: http")

In [6]:
import boto3
from botocore.exceptions import NoCredentialsError

# Your MinIO/S3 credentials and bucket details
endpoint_url = 'http://localhost:9000'  # Your MinIO endpoint
access_key = 'minio_debug'
secret_key = 'minio123'
bucket_name = 'depictio-bucket'
object_key = '64a842842bf4fa7deaa3dbed'  # The name of the file you want to read

# Create an S3 client
s3_client = boto3.client('s3',
                         endpoint_url=endpoint_url,
                         aws_access_key_id=access_key,
                         aws_secret_access_key=secret_key,
                         region_name='us-east-1')  # Region name can be any string


In [ ]:
try:
    file_content = s3_client.get_object(Bucket=bucket_name, Key=object_key)['Body'].read()
    print(file_content.decode('utf-8'))  # Assuming the file is text-based; decode as necessary
except NoCredentialsError:
    print("Credentials not available")
except Exception as e:
    print(e)


In [19]:
from io import BytesIO
import boto3
import polars as pl

# MinIO/S3 credentials and bucket details
endpoint_url = 'http://localhost:9000'  # Your MinIO endpoint
access_key = 'minio'
secret_key = 'minio123'
bucket_name = 'depictio-bucket'
object_key = '64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/1-ea34b2f8-4b6c-455f-b3ca-1ae0519c792f-0.parquet'  # Name of your parquet file

# Create S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=endpoint_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)

# Access object using S3 client
obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)

# Retrieve the Parquet file as a bytes stream
try:
    obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    
    # Use BytesIO to convert bytes data to a file-like object for pandas
    data_stream = BytesIO(data)
    
    # Use pandas to read the Parquet file
    df = pl.read_parquet(data_stream)
    print(df)
except boto3.exceptions.Boto3Error as e:
    print(f"An error occurred: {e}")

shape: (6_720, 15)
┌──────────────┬────────────┬─────────────┬────────┬───┬──────────┬───────────┬──────┬─────────────┐
│ depictio_run ┆ sample     ┆ cell        ┆ medbin ┆ … ┆ nb_p     ┆ nb_r      ┆ nb_a ┆ bam         │
│ _id          ┆ ---        ┆ ---         ┆ ---    ┆   ┆ ---      ┆ ---       ┆ ---  ┆ ---         │
│ ---          ┆ str        ┆ str         ┆ i64    ┆   ┆ f64      ┆ f64       ┆ f64  ┆ str         │
│ str          ┆            ┆             ┆        ┆   ┆          ┆           ┆      ┆             │
╞══════════════╪════════════╪═════════════╪════════╪═══╪══════════╪═══════════╪══════╪═════════════╡
│ 2019-09-11-H ┆ HG01114x02 ┆ HG01114x02P ┆ 23     ┆ … ┆ 0.008158 ┆ 0.196344  ┆ 0.1  ┆ /scratch/tw │
│ TN7CAFXY     ┆            ┆ E20301      ┆        ┆   ┆          ┆           ┆      ┆ eber/DATA/M │
│              ┆            ┆             ┆        ┆   ┆          ┆           ┆      ┆ C_DATA/STO… │
│ 2019-09-11-H ┆ HG01114x02 ┆ HG01114x02P ┆ 5      ┆ … ┆ 0.008158 ┆ 0.05

In [2]:
from io import BytesIO
import boto3
import polars as pl
import os

# Set environment variables for MinIO access
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_ENDPOINT_URL"] = "http://localhost:9000"  # This might need to be AWS_S3_ENDPOINT_URL depending on the version and setup
os.environ["AWS_REGION"] = "us-east-1"  # This can be a placeholder value


object_key = 's3://depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/1-ea34b2f8-4b6c-455f-b3ca-1ae0519c792f-0.parquet'  # Name of your parquet file


# Create S3 client
# s3_client = boto3.client(
#     's3',
#     endpoint_url=endpoint_url,
#     aws_access_key_id=access_key,
#     aws_secret_access_key=secret_key
# )

# # Access object using S3 client
# obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)

# Retrieve the Parquet file as a bytes stream
try:
    # obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)
    # data = obj['Body'].read()
    
    # # Use BytesIO to convert bytes data to a file-like object for pandas
    # data_stream = BytesIO(data)
    
    # Use pandas to read the Parquet file
    df = pl.read_parquet(object_key)
    print(df)
except boto3.exceptions.Boto3Error as e:
    print(f"An error occurred: {e}")

shape: (6_720, 15)
┌──────────────┬────────────┬─────────────┬────────┬───┬──────────┬───────────┬──────┬─────────────┐
│ depictio_run ┆ sample     ┆ cell        ┆ medbin ┆ … ┆ nb_p     ┆ nb_r      ┆ nb_a ┆ bam         │
│ _id          ┆ ---        ┆ ---         ┆ ---    ┆   ┆ ---      ┆ ---       ┆ ---  ┆ ---         │
│ ---          ┆ str        ┆ str         ┆ i64    ┆   ┆ f64      ┆ f64       ┆ f64  ┆ str         │
│ str          ┆            ┆             ┆        ┆   ┆          ┆           ┆      ┆             │
╞══════════════╪════════════╪═════════════╪════════╪═══╪══════════╪═══════════╪══════╪═════════════╡
│ 2019-09-11-H ┆ HG01114x02 ┆ HG01114x02P ┆ 23     ┆ … ┆ 0.008158 ┆ 0.196344  ┆ 0.1  ┆ /scratch/tw │
│ TN7CAFXY     ┆            ┆ E20301      ┆        ┆   ┆          ┆           ┆      ┆ eber/DATA/M │
│              ┆            ┆             ┆        ┆   ┆          ┆           ┆      ┆ C_DATA/STO… │
│ 2019-09-11-H ┆ HG01114x02 ┆ HG01114x02P ┆ 5      ┆ … ┆ 0.008158 ┆ 0.05

In [4]:
import os
import polars as pl

# Set environment variables for MinIO access
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_ENDPOINT_URL"] = "http://localhost:9000"  # This might need to be AWS_S3_ENDPOINT_URL depending on the version and setup
os.environ["AWS_REGION"] = "us-east-1"  # This can be a placeholder value

# Assuming the Delta table is located at this path in your MinIO bucket
delta_table_path = "s3://depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/"

# Attempt to read the Delta table using Polars
try:
    df = pl.read_delta(delta_table_path)
    print(df)
except Exception as e:
    print(f"Failed to read Delta table: {e}")


Failed to read Delta table: Generic S3 error: Error after 0 retries in 2.458µs, max_retries:10, retry_timeout:180s, source:builder error for url (http://localhost:9000/depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/_delta_log/_last_checkpoint): URL scheme is not allowed


In [ ]:
import os
import polars as pl

# Set environment variables for MinIO access
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:9000"  # This might need to be AWS_S3_ENDPOINT_URL depending on the version and setup
os.environ["AWS_REGION"] = "us-east-1"  # This can be a placeholder value

# Assuming the Delta table is located at this path in your MinIO bucket
delta_table_path = "s3://depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/"

# Attempt to read the Delta table using Polars
try:
    df = pl.read_delta(delta_table_path)
    print(df)
except Exception as e:
    print(f"Failed to read Delta table: {e}")


Failed to read Delta table: Generic S3 error: Error after 0 retries in 2.166µs, max_retries:10, retry_timeout:180s, source:builder error for url (http://localhost:9000/depictio-bucket/64a842842bf4fa7deaa3dbed/65c4ef0f26d99f46c993a5af/65c4ef0f26d99f46c993a5b0/_delta_log/_last_checkpoint): URL scheme is not allowed
